# Microsoft Agent Framework ile İnsan-Döngüsü İş Akışı

## 🎯 Öğrenme Hedefleri

Bu not defterinde, Microsoft Agent Framework'ün `RequestInfoExecutor` özelliğini kullanarak **insan-döngüsü** iş akışlarını nasıl uygulayacağınızı öğreneceksiniz. Bu güçlü yöntem, yapay zeka iş akışlarını duraklatıp insan girdisi almanıza olanak tanır, böylece ajanlarınızı etkileşimli hale getirir ve kritik kararlar üzerinde insan kontrolü sağlar.

## 🔄 İnsan-Döngüsü Nedir?

**İnsan-döngüsü (HITL)**, yapay zeka ajanlarının devam etmeden önce insan girdisi talep etmek için yürütmeyi duraklattığı bir tasarım modelidir. Bu model şu durumlar için önemlidir:

- ✅ **Kritik kararlar** - Önemli eylemler gerçekleştirmeden önce insan onayı alın
- ✅ **Belirsiz durumlar** - Yapay zeka kararsız olduğunda insanlara açıklama yapma fırsatı verin
- ✅ **Kullanıcı tercihleri** - Kullanıcılardan birden fazla seçenek arasında seçim yapmalarını isteyin
- ✅ **Uyumluluk ve güvenlik** - Düzenlemeye tabi operasyonlar için insan denetimini sağlayın
- ✅ **Etkileşimli deneyimler** - Kullanıcı girdisine yanıt veren konuşma ajanları oluşturun

## 🏗️ Microsoft Agent Framework'te Nasıl Çalışır?

Framework, insan-döngüsü için üç ana bileşen sağlar:

1. **`RequestInfoExecutor`** - İş akışını duraklatan ve bir `RequestInfoEvent` oluşturan özel bir yürütücü
2. **`RequestInfoMessage`** - İnsanlara gönderilen türlendirilmiş istek yükleri için temel sınıf
3. **`RequestResponse`** - İnsan yanıtlarını orijinal isteklerle `request_id` kullanarak ilişkilendirir

**İş Akışı Modeli:**
```
Agent detects need for input
    ↓
Sends message to RequestInfoExecutor
    ↓
Workflow pauses & emits RequestInfoEvent
    ↓
Application collects human input (console, UI, etc.)
    ↓
Application sends RequestResponse via send_responses_streaming()
    ↓
Workflow resumes with human input
```

## 🏨 Örneğimiz: Kullanıcı Onayı ile Otel Rezervasyonu

Alternatif destinasyonlar önermeden **önce** insan onayı ekleyerek koşullu iş akışını geliştireceğiz:

1. Kullanıcı bir destinasyon talep eder (örneğin, "Paris")
2. `availability_agent` odaların müsait olup olmadığını kontrol eder
3. **Eğer oda yoksa** → `confirmation_agent` "Alternatifleri görmek ister misiniz?" diye sorar
4. İş akışı **`RequestInfoExecutor`** kullanılarak duraklatılır
5. **İnsan yanıt verir**: Konsol girişi üzerinden "evet" veya "hayır"
6. `decision_manager` yanıtı yönlendirir:
   - **Evet** → Alternatif destinasyonları göster
   - **Hayır** → Rezervasyon talebini iptal et
7. Nihai sonucu göster

Bu örnek, kullanıcıların ajanın önerileri üzerinde kontrol sahibi olmasını nasıl sağlayabileceğinizi gösteriyor!

---

Haydi başlayalım! 🚀


## Adım 1: Gerekli Kütüphaneleri İçe Aktarma

Standart Agent Framework bileşenlerini ve **insan müdahalesi gerektiren özel sınıfları** içe aktarıyoruz:
- `RequestInfoExecutor` - İnsan girdisi için iş akışını duraklatan yürütücü
- `RequestInfoEvent` - İnsan girdisi istendiğinde yayımlanan olay
- `RequestInfoMessage` - Türlendirilmiş istek yükleri için temel sınıf
- `RequestResponse` - İnsan yanıtlarını isteklerle ilişkilendirir
- `WorkflowOutputEvent` - İş akışı çıktısını algılamak için olay


In [21]:
import asyncio
import json
import os
from dataclasses import dataclass
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Executor,
    RequestInfoEvent,          # NEW: Event when human input is requested
    RequestInfoExecutor,       # NEW: Executor that gathers human input
    RequestInfoMessage,        # NEW: Base class for request payloads
    RequestResponse,           # NEW: Correlates response with request
    Role,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,       # NEW: Event for workflow outputs
    WorkflowRunState,          # NEW: Enum of workflow run states
    WorkflowStatusEvent,       # NEW: Event for run state changes
    ai_function,
    executor,
    handler,                   # NEW: Decorator for executor methods
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")
print("🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse")

✅ All imports successful!
🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse


## Adım 2: Yapılandırılmış Çıktılar için Pydantic Modellerini Tanımlayın

Bu modeller, ajanların döndüreceği **şemayı** tanımlar. Koşullu iş akışından gelen tüm modelleri koruyoruz ve şunları ekliyoruz:

**İnsan Döngüsü İçin Yeni:**
- `HumanFeedbackRequest` - İnsanlara gönderilen istek yükünü tanımlayan `RequestInfoMessage` alt sınıfı
  - `prompt` (sorulacak soru) ve `destination` (mevcut olmayan şehir hakkında bağlam) içerir


In [22]:
# Existing models from conditional workflow
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""
    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""
    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""
    destination: str
    action: str
    message: str


# NEW: Pydantic model for agent's response format
class ConfirmationQuestion(BaseModel):
    """
    Pydantic model used by confirmation_agent's response_format.
    This is what the agent will output as JSON.
    """
    question: str  # The question to ask the user
    destination: str  # The unavailable destination for context


# NEW: Dataclass for RequestInfoExecutor
@dataclass
class HumanFeedbackRequest(RequestInfoMessage):
    """
    Request sent to RequestInfoExecutor asking if user wants alternatives.
    
    MUST be a dataclass subclassing RequestInfoMessage for type compatibility.
    This is what gets sent to the RequestInfoExecutor.
    """
    prompt: str = ""  # The question to ask the user
    destination: str = ""  # The unavailable destination for context


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")
print("   - ConfirmationQuestion (agent response format) 🆕")
print("   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)
   - ConfirmationQuestion (agent response format) 🆕
   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕


## Adım 3: Otel Rezervasyon Aracını Oluşturun

Koşullu iş akışından aynı araç - varış noktasında odaların müsait olup olmadığını kontrol eder.


In [23]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Adım 4: Yönlendirme için Koşul Fonksiyonlarını Tanımlayın

İnsan müdahalesi gerektiren iş akışımız için **dört koşul fonksiyonuna** ihtiyacımız var:

**Koşullu iş akışından:**
1. `has_availability_condition` - Oteller MEVCUT olduğunda yönlendirir
2. `no_availability_condition` - Oteller MEVCUT OLMADIĞINDA yönlendirir

**İnsan müdahalesi gerektiren iş akışı için yeni:**
3. `user_wants_alternatives_condition` - Kullanıcı alternatiflere "evet" dediğinde yönlendirir
4. `user_declines_alternatives_condition` - Kullanıcı alternatiflere "hayır" dediğinde yönlendirir


In [24]:
# Existing condition functions from conditional workflow
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )
        return result.has_availability
    except Exception as e:
        display(HTML(f"""<div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'><strong>⚠️  Error:</strong> {str(e)}</div>"""))
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )
        return not result.has_availability
    except Exception as e:
        return False


# NEW: Condition functions for human-in-the-loop routing
def user_wants_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user WANTS to see alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            wants_alternatives = "wants to see alternative" in msg_text or "want to see alternative" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
                    <strong>🔍 User Decision:</strong> User wants alternatives = <strong>{wants_alternatives}</strong>
                </div>
            """)
            )
            
            return wants_alternatives
    
    return False
def user_declines_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user DECLINES alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            declined = "declined" in msg_text or "has declined" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fce4ec; border-left: 4px solid #c2185b; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 User Decision:</strong> User declined alternatives = <strong>{declined}</strong>
                </div>
            """)
            )
            
            return declined
    
    return False
print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")
print("   - user_wants_alternatives_condition (routes when user says yes) 🆕")
print("   - user_declines_alternatives_condition (routes when user says no) 🆕")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)
   - user_wants_alternatives_condition (routes when user says yes) 🆕
   - user_declines_alternatives_condition (routes when user says no) 🆕


## Adım 5: Karar Yöneticisi Yürütücüsünü Oluşturun

Bu, **insan döngüsünün merkezidir**! `DecisionManager`, aşağıdaki işlevleri yerine getiren özel bir `Executor`:

1. **İnsan geri bildirimini alır** `RequestResponse` nesneleri aracılığıyla
2. **Kullanıcının kararını işler** (evet/hayır)
3. **İş akışını yönlendirir** uygun ajanlara mesajlar göndererek

Ana özellikler:
- Yöntemleri iş akışı adımları olarak açığa çıkarmak için `@handler` dekoratörünü kullanır
- Hem orijinal isteği hem de kullanıcının cevabını içeren `RequestResponse[HumanFeedbackRequest, str]` alır
- Koşul fonksiyonlarımızı tetikleyen basit "evet" veya "hayır" mesajları üretir


In [25]:
class DecisionManager(Executor):
    """
    Coordinates workflow routing based on human feedback.
    
    This executor receives RequestResponse objects from the RequestInfoExecutor
    and makes routing decisions by sending simple messages that trigger
    condition functions.
    """

    def __init__(self, id: str | None = None):
        super().__init__(id=id or "decision_manager")

    @handler
    async def on_human_feedback(
        self,
        feedback: RequestResponse[HumanFeedbackRequest, str],
        ctx: WorkflowContext[AgentExecutorRequest],
    ) -> None:
        """
        Process human feedback and let the workflow route based on conditions.
        
        The RequestResponse contains:
        - feedback.data: The user's string reply (e.g., "yes" or "no")
        - feedback.original_request: The HumanFeedbackRequest with context
        
        This handler just displays feedback and passes the RequestResponse through.
        The routing is done by condition functions on the edges.
        """
        user_reply = (feedback.data or "").strip().lower()
        destination = getattr(feedback.original_request, "destination", "unknown")

        display(
            HTML(f"""
            <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
                <strong>🎯 Decision Manager:</strong> Processing user reply: <strong>"{user_reply}"</strong> for {destination}
            </div>
        """)
        )

        if user_reply == "yes":
            display(
                HTML("""
                <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                    <strong>➡️  Routing:</strong> User wants alternatives → Will route to alternative_agent
                </div>
            """)
            )
            # Create and send a message for the alternative_agent
            user_msg = ChatMessage(
                Role.USER,
                text=f"The user wants to see alternative destinations near {destination}. Please suggest one.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        elif user_reply == "no":
            display(
                HTML("""
                <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 Routing:</strong> User declined alternatives → Will route to cancellation_agent
                </div>
            """)
            )
            # Create and send a message for the cancellation_agent
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        else:
            # Handle unexpected input - treat as decline
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                    <strong>⚠️  Warning:</strong> Unexpected input "{user_reply}" - treating as decline
                </div>
            """)
            )
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))


print("✅ DecisionManager executor created with @handler method for human feedback")

✅ DecisionManager executor created with @handler method for human feedback


## Adım 6: Özel Görüntüleme Yürütücüsü Oluşturun

Koşullu iş akışından aynı görüntüleme yürütücüsü - iş akışı çıktısı olarak nihai sonuçları verir.


In [26]:
@executor(id="prepare_human_request")
async def prepare_human_request(
    response: AgentExecutorResponse, 
    ctx: WorkflowContext[HumanFeedbackRequest]
) -> None:
    """
    Transform agent response into HumanFeedbackRequest for RequestInfoExecutor.
    
    This executor bridges the type gap between:
    - confirmation_agent outputs AgentExecutorResponse with ConfirmationQuestion JSON
    - request_info_executor expects HumanFeedbackRequest (RequestInfoMessage dataclass)
    """
    display(
        HTML("""
        <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Transform:</strong> Converting ConfirmationQuestion to HumanFeedbackRequest
        </div>
    """)
    )
    
    # Parse the agent's Pydantic output (ConfirmationQuestion)
    confirmation = ConfirmationQuestion.model_validate_json(response.agent_run_response.text)
    
    # Convert to HumanFeedbackRequest dataclass for RequestInfoExecutor
    feedback_request = HumanFeedbackRequest(
        prompt=confirmation.question,
        destination=confirmation.destination
    )
    
    # Send the properly typed RequestInfoMessage to the RequestInfoExecutor
    await ctx.send_message(feedback_request)


@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ prepare_human_request executor created with @executor decorator")
print("✅ display_result executor created with @executor decorator")

✅ prepare_human_request executor created with @executor decorator
✅ display_result executor created with @executor decorator


## Adım 7: Ortam Değişkenlerini Yükle

LLM istemcisini yapılandırın (GitHub Models, Azure OpenAI veya OpenAI).


In [27]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(
    base_url=os.environ.get("GITHUB_ENDPOINT"), 
    api_key=os.environ.get("GITHUB_TOKEN"), 
    model_id="gpt-4o"
)

print("✅ Chat client configured with GitHub Models")

✅ Chat client configured with GitHub Models


## Adım 8: AI Ajanları ve Yürütücüler Oluşturma

**Altı iş akışı bileşeni** oluşturuyoruz:

**Ajanlar (AgentExecutor içinde sarılmış):**
1. **availability_agent** - Araç kullanarak otel müsaitliğini kontrol eder
2. **confirmation_agent** - 🆕 İnsan onay talebini hazırlar
3. **alternative_agent** - Alternatif şehirler önerir (kullanıcı evet dediğinde)
4. **booking_agent** - Rezervasyonu teşvik eder (odalar müsait olduğunda)
5. **cancellation_agent** - 🆕 İptal mesajını işler (kullanıcı hayır dediğinde)

**Özel Yürütücüler:**
6. **request_info_executor** - 🆕 `RequestInfoExecutor`, iş akışını insan girdisi için duraklatır
7. **decision_manager** - 🆕 İnsan yanıtına göre yönlendiren özel yürütücü (yukarıda zaten tanımlandı)


In [28]:
# Agent 1: Check availability with tool (same as conditional workflow)
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: NEW - Prepare human confirmation request
confirmation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user's requested destination has no available hotel rooms. "
            "Create a polite message asking if they would like to see alternative destinations nearby. "
            "Return a JSON with: destination (the unavailable city), and question (a friendly yes/no question). "
            "Keep the question concise and friendly."
        ),
        response_format=ConfirmationQuestion,  # Use Pydantic model for agent output
    ),
    id="confirmation_agent",
)

# Agent 3: Suggest alternative (when user says yes)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 4: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

# Agent 5: NEW - Handle cancellation when user declines alternatives
class CancellationMessage(BaseModel):
    """Message when user declines alternatives."""
    status: str
    message: str

cancellation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user has declined to see alternative hotel destinations. "
            "Create a polite cancellation message. "
            "Return JSON with: status (should be 'cancelled'), and message (a friendly acknowledgment). "
            "Keep the message brief and understanding."
        ),
        response_format=CancellationMessage,
    ),
    id="cancellation_agent",
)

# NEW: RequestInfoExecutor - pauses workflow to gather human input
request_info_executor = RequestInfoExecutor(id="request_info")

# NEW: DecisionManager instance - routes based on human feedback
decision_manager = DecisionManager(id="decision_manager")

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created Workflow Components:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>confirmation_agent</strong> 🆕 - Prepares human confirmation request</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
            <li><strong>cancellation_agent</strong> 🆕 - Handles user declining alternatives</li>
            <li><strong>request_info_executor</strong> 🆕 - Pauses workflow for human input</li>
            <li><strong>decision_manager</strong> 🆕 - Routes based on human response</li>
        </ul>
    </div>
""")
)

## Adım 9: İnsan Müdahalesi ile İş Akışını Oluşturma

Şimdi, insan müdahalesi yolunu içeren **koşullu yönlendirme** ile iş akışı grafiğini oluşturuyoruz:

**İş Akışı Yapısı:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙                    ↘
[no_availability]        [has_availability]
        ↓                        ↓
confirmation_agent          booking_agent
        ↓                        ↓
prepare_human_request      display_result
        ↓
request_info_executor (PAUSE)
        ↓
decision_manager
   ↙         ↘
[yes]        [no]
   ↓           ↓
alternative  cancellation
   ↓           ↓
display_result
```

**Ana Bağlantılar:**
- `availability_agent → confirmation_agent` (odalar yoksa)
- `confirmation_agent → prepare_human_request` (dönüşüm türü)
- `prepare_human_request → request_info_executor` (insan müdahalesi için duraklama)
- `request_info_executor → decision_manager` (her zaman - RequestResponse sağlar)
- `decision_manager → alternative_agent` (kullanıcı "evet" dediğinde)
- `decision_manager → cancellation_agent` (kullanıcı "hayır" dediğinde)
- `availability_agent → booking_agent` (odalar mevcutsa)
- Tüm yollar `display_result` ile sonlanır


In [29]:
# Build the workflow with human-in-the-loop routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    
    # NO AVAILABILITY PATH (with human-in-the-loop)
    .add_edge(availability_agent, confirmation_agent, condition=no_availability_condition)
    .add_edge(confirmation_agent, prepare_human_request)  # Transform to HumanFeedbackRequest
    .add_edge(prepare_human_request, request_info_executor)  # Send to RequestInfoExecutor
    .add_edge(request_info_executor, decision_manager)    # Always goes to decision manager
    
    # Decision manager routes based on user response
    .add_edge(decision_manager, alternative_agent, condition=user_wants_alternatives_condition)
    .add_edge(decision_manager, cancellation_agent, condition=user_declines_alternatives_condition)
    .add_edge(alternative_agent, display_result)
    .add_edge(cancellation_agent, display_result)
    
    # HAS AVAILABILITY PATH (no human input needed)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Human-in-the-Loop Routing:</strong><br>
            • If <strong>NO availability</strong> → confirmation_agent → prepare_human_request → request_info_executor → <strong>PAUSE FOR HUMAN</strong> → decision_manager<br>
            &nbsp;&nbsp;• If user says <strong>YES</strong> → alternative_agent → display_result<br>
            &nbsp;&nbsp;• If user says <strong>NO</strong> → cancellation_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result (no human input needed)
        </p>
    </div>
""")
)

## Adım 10: Test Durumu 1'i Çalıştır - Uygunluk OLMAYAN Şehir (Paris, İnsan Onayı ile)

Bu test, **tam insan-döngüsü sürecini** gösterir:

1. Paris'te otel talep edin
2. availability_agent kontrol eder → Hiç oda yok
3. confirmation_agent insan odaklı bir soru oluşturur
4. request_info_executor **iş akışını duraklatır** ve `RequestInfoEvent` yayar
5. **Uygulama olayı algılar ve konsolda kullanıcıya sorar**
6. Kullanıcı "evet" veya "hayır" yazar
7. Uygulama yanıtı `send_responses_streaming()` aracılığıyla gönderir
8. decision_manager yanıtı temel alarak yönlendirme yapar
9. Nihai sonuç görüntülenir

**Ana Kalıp:**
- İlk iterasyon için `workflow.run_stream()` kullanın
- Sonraki iterasyonlar için `workflow.send_responses_streaming(pending_responses)` kullanın
- İnsan girdisine ihtiyaç duyulduğunda algılamak için `RequestInfoEvent` dinleyin
- Nihai sonuçları yakalamak için `WorkflowOutputEvent` dinleyin


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability - Human-in-the-Loop)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → confirmation_agent → request_info_executor → <strong>PAUSE</strong> → decision_manager → (depends on user input)</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], 
    should_respond=True
)

# Human-in-the-loop execution pattern
pending_responses: dict[str, str] | None = None
completed = False
workflow_output: str | None = None

print("\n🔄 Starting human-in-the-loop workflow...")
print("=" * 60)

while not completed:
    # First iteration uses run_stream with the request
    # Subsequent iterations use send_responses_streaming with collected human responses
    if pending_responses:
        print(f"\n📤 Sending human responses: {pending_responses}")
        stream = workflow.send_responses_streaming(pending_responses)
        pending_responses = None  # Clear immediately after sending
    else:
        print(f"\n🚀 Starting workflow with request: 'I want to book a hotel in Paris'")
        stream = workflow.run_stream(request_paris)
    
    # Collect all events from this iteration
    events = [event async for event in stream]
    
    # Process events
    requests: list[tuple[str, str]] = []  # (request_id, prompt)
    
    for event in events:
        # Check for human input requests
        if isinstance(event, RequestInfoEvent) and isinstance(event.data, HumanFeedbackRequest):
            print(f"\n⏸️  WORKFLOW PAUSED - Human input requested!")
            print(f"   Request ID: {event.request_id}")
            print(f"   Destination: {event.data.destination}")
            requests.append((event.request_id, event.data.prompt))
        
        # Check for workflow outputs
        elif isinstance(event, WorkflowOutputEvent):
            workflow_output = str(event.data)
            completed = True
            print(f"\n✅ Workflow completed with output!")
    
    # If we have human requests, prompt the user
    if requests and not completed:
        responses: dict[str, str] = {}
        for req_id, prompt in requests:
            print(f"\n{'='*60}")
            print(f"💬 QUESTION FOR YOU:")
            print(f"   {prompt}")
            print(f"{'='*60}")
            
            # Get user input (in notebook, this will pause execution)
            answer = input("👉 Enter 'yes' or 'no': ").strip().lower()
            
            print(f"\n📝 You answered: {answer}")
            responses[req_id] = answer
        
        pending_responses = responses

print(f"\n{'='*60}")
print(f"🏆 FINAL WORKFLOW OUTPUT:")
print(f"{'='*60}")

# Display final result
if workflow_output:
    # Try to parse as JSON for pretty display
    try:
        result_data = json.loads(workflow_output)
        if "alternative_destination" in result_data:
            result_obj = AlternativeResult.model_validate_json(workflow_output)
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> ✅ Accepted alternatives</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_obj.alternative_destination}</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_obj.reason}</p>
                    </div>
                </div>
            """)
            )
        else:
            # User declined
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #f44336 0%, #e91e63 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(244,67,54,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> 🚫 Declined alternatives</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Result:</strong> Booking request cancelled</p>
                    </div>
                </div>
            """)
            )
    except:
        print(workflow_output)


🔄 Starting human-in-the-loop workflow...

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: 032c8fce-b9d1-400e-ba8d-afd2248e2926
   Destination: Paris

💬 QUESTION FOR YOU:
   Unfortunately, there are no rooms available in Paris. Would you like to explore nearby alternative destinations?

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: cf48dad0-ee5e-4f60-8806-341a7a292bd4
   Destination: Paris

💬 QUESTION FOR YOU:
   I'm sorry to inform you that there are no available hotel rooms in Paris. Would you like me to suggest nearby alternative destinations?

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'


## Adım 11: Test Durumu 2'yi Çalıştır - Müsaitlik Olan Şehir (Stockholm - İnsan Girdisi Gerekmez)

Bu test, odaların müsait olduğu **doğrudan yolu** gösterir:

1. Stockholm'de otel talep edin
2. availability_agent kontrol eder → Odalar müsait ✅
3. booking_agent rezervasyon önerir
4. display_result onay gösterir
5. **İnsan girdisi gerekmez!**

Odalar müsait olduğunda iş akışı tamamen insan müdahalesi gerektiren yolu atlar.


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability - No Human Input)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result (direct, no pause)</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], 
    should_respond=True
)

# Run the workflow (should complete without human input)
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm - No Human Input)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0 0 10px 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <p style='margin: 10px 0 0 0; font-size: 12px; color: #999; font-style: italic;'>Note: No human input was requested because rooms were available!</p>
            </div>
        </div>
    """)
    )

## Temel Çıkarımlar ve İnsan Müdahalesiyle En İyi Uygulamalar

### ✅ Öğrendikleriniz:

#### 1. **RequestInfoExecutor Deseni**
Microsoft Agent Framework'deki insan müdahalesi deseni üç ana bileşen kullanır:
- `RequestInfoExecutor` - İş akışını duraklatır ve olaylar yayar
- `RequestInfoMessage` - Türlendirilmiş istek yükleri için temel sınıf (bunu alt sınıf yapın!)
- `RequestResponse` - İnsan yanıtlarını orijinal isteklerle ilişkilendirir

**Kritik Anlayış:**
- `RequestInfoExecutor` kendi başına giriş toplamaz - yalnızca iş akışını duraklatır
- Uygulama kodunuzun `RequestInfoEvent` dinlemesi ve giriş toplaması gerekir
- Kullanıcının yanıtını `request_id` ile eşleştiren bir sözlükle `send_responses_streaming()` çağırmanız gerekir

#### 2. **Akışlı Çalıştırma Deseni**
```python
# First iteration
stream = workflow.run_stream(initial_request)

# Subsequent iterations (after human input)
stream = workflow.send_responses_streaming(pending_responses)

# Always process events
events = [event async for event in stream]
```

#### 3. **Olay Tabanlı Mimari**
İş akışını kontrol etmek için belirli olayları dinleyin:
- `RequestInfoEvent` - İnsan girdisi gerekli (iş akışı duraklatıldı)
- `WorkflowOutputEvent` - Nihai sonuç mevcut (iş akışı tamamlandı)
- `WorkflowStatusEvent` - Durum değişiklikleri (IN_PROGRESS, IDLE_WITH_PENDING_REQUESTS, vb.)

#### 4. **@handler ile Özel Yürütücüler**
`DecisionManager`, aşağıdaki özelliklere sahip yürütücüler oluşturmayı gösterir:
- Yöntemleri iş akışı adımları olarak açığa çıkarmak için `@handler` dekoratörünü kullanır
- Türlendirilmiş mesajlar alır (ör. `RequestResponse[HumanFeedbackRequest, str]`)
- İş akışını diğer yürütücülere mesaj göndererek yönlendirir
- `WorkflowContext` aracılığıyla bağlama erişir

#### 5. **İnsan Kararlarıyla Koşullu Yönlendirme**
İnsan yanıtlarını değerlendiren koşul fonksiyonları oluşturabilirsiniz:
```python
def user_wants_alternatives_condition(message: Any) -> bool:
    response_text = message.agent_run_response.text.lower()
    return response_text == "yes"
```

### 🎯 Gerçek Dünya Uygulamaları:

1. **Onay İş Akışları**
   - Harcama raporlarını işlemeye başlamadan önce yönetici onayı alın
   - Otomatik e-postaları göndermeden önce insan incelemesi gerektirir
   - Yüksek değerli işlemleri gerçekleştirmeden önce onay alın

2. **İçerik Moderasyonu**
   - Şüpheli içeriği insan incelemesi için işaretleyin
   - Moderatörlerden sınır durumlarında nihai karar vermelerini isteyin
   - AI güveni düşük olduğunda insanlara yönlendirin

3. **Müşteri Hizmetleri**
   - AI'nın rutin soruları otomatik olarak ele almasına izin verin
   - Karmaşık sorunları insan temsilcilere yönlendirin
   - Müşteriye bir insanla konuşmak isteyip istemediğini sorun

4. **Veri İşleme**
   - İnsanlardan belirsiz veri girişlerini çözmelerini isteyin
   - AI'nın belirsiz belgeleri yorumlamasını doğrulayın
   - Kullanıcıların birden fazla geçerli yorum arasında seçim yapmasına izin verin

5. **Güvenlik Kritik Sistemler**
   - Geri dönüşü olmayan işlemlerden önce insan onayı gerektirir
   - Hassas verilere erişmeden önce onay alın
   - Düzenlenmiş sektörlerde (sağlık, finans) kararları doğrulayın

6. **Etkileşimli Ajanlar**
   - Takip soruları soran sohbet botları oluşturun
   - Kullanıcıları karmaşık süreçlerde yönlendiren sihirbazlar tasarlayın
   - İnsanlarla adım adım işbirliği yapan ajanlar tasarlayın

### 🔄 Karşılaştırma: İnsan Müdahalesiyle ve İnsan Müdahalesi Olmadan

| Özellik | Koşullu İş Akışı | İnsan Müdahalesiyle İş Akışı |
|---------|------------------|-----------------------------|
| **Çalıştırma** | Tek `workflow.run()` | Döngü ile `run_stream()` + `send_responses_streaming()` |
| **Kullanıcı Girdisi** | Yok (tam otomatik) | `input()` veya UI aracılığıyla etkileşimli istemler |
| **Bileşenler** | Ajanlar + Yürütücüler | + RequestInfoExecutor + DecisionManager |
| **Olaylar** | Sadece AgentExecutorResponse | RequestInfoEvent, WorkflowOutputEvent, vb. |
| **Duraklatma** | Duraklatma yok | İş akışı RequestInfoExecutor'da duraklatılır |
| **İnsan Kontrolü** | İnsan kontrolü yok | İnsanlar önemli kararlar alır |
| **Kullanım Durumu** | Otomasyon | İşbirliği ve denetim |

### 🚀 İleri Düzey Desenler:

#### Birden Fazla İnsan Karar Noktası
Aynı iş akışında birden fazla `RequestInfoExecutor` düğümüne sahip olabilirsiniz:
```python
.add_edge(agent1, request_info_1)  # First human decision
.add_edge(decision_manager_1, agent2)
.add_edge(agent2, request_info_2)  # Second human decision
.add_edge(decision_manager_2, final_agent)
```

#### Zaman Aşımı Yönetimi
İnsan yanıtları için zaman aşımı uygulayın:
```python
import asyncio

try:
    answer = await asyncio.wait_for(
        asyncio.to_thread(input, "Enter yes/no: "),
        timeout=60.0
    )
except asyncio.TimeoutError:
    answer = "no"  # Default to safe option
```

#### Zengin UI Entegrasyonu
`input()` yerine web UI, Slack, Teams, vb. ile entegre edin:
```python
if isinstance(event, RequestInfoEvent):
    # Send notification to user's preferred channel
    await slack_client.send_message(
        user_id=current_user,
        text=event.data.prompt,
        request_id=event.request_id
    )
```

#### Koşullu İnsan Müdahalesi
Sadece belirli durumlarda insan girdisi isteyin:
```python
def needs_human_approval_condition(message: Any) -> bool:
    # Only route to human if confidence is low or value is high
    if result.confidence < 0.7 or result.value > 10000:
        return True
    return False
```

### ⚠️ En İyi Uygulamalar:

1. **Her Zaman RequestInfoMessage'ı Alt Sınıf Yapın**
   - Tür güvenliği ve doğrulama sağlar
   - UI oluşturma için zengin bağlam sağlar
   - Her istek türünün amacını netleştirir

2. **Açıklayıcı İstemler Kullanın**
   - Ne istediğiniz hakkında bağlam sağlayın
   - Her seçimin sonuçlarını açıklayın
   - Soruları basit ve net tutun

3. **Beklenmeyen Girdiyi Ele Alın**
   - Kullanıcı yanıtlarını doğrulayın
   - Geçersiz girişler için varsayılanlar sağlayın
   - Net hata mesajları verin

4. **İstek Kimliklerini Takip Edin**
   - request_id ile yanıtlar arasındaki ilişkiyi kullanın
   - Durumu manuel olarak yönetmeye çalışmayın

5. **Engellemeyen Tasarım Yapın**
   - Giriş beklerken iş parçacıklarını engellemeyin
   - Tüm süreçte asenkron desenler kullanın
   - Eşzamanlı iş akışı örneklerini destekleyin

### 📚 İlgili Kavramlar:

- **Ajan Ara Yazılımı** - Ajan çağrılarını engelleyin (önceki not defteri)
- **İş Akışı Durum Yönetimi** - Çalıştırmalar arasında iş akışı durumunu koruyun
- **Çoklu Ajan İşbirliği** - İnsan müdahalesini ajan ekipleriyle birleştirin
- **Olay Tabanlı Mimari** - Olaylarla reaktif sistemler oluşturun

---

### 🎓 Tebrikler!

Microsoft Agent Framework ile insan müdahalesi iş akışlarını ustalıkla öğrendiniz! Artık şunları biliyorsunuz:
- ✅ İnsan girdisi toplamak için iş akışlarını duraklatma
- ✅ RequestInfoExecutor ve RequestInfoMessage kullanma
- ✅ Olaylarla akışlı çalıştırmayı ele alma
- ✅ @handler ile özel yürütücüler oluşturma
- ✅ İnsan kararlarına dayalı iş akışlarını yönlendirme
- ✅ İnsanlarla işbirliği yapan etkileşimli AI ajanları oluşturma

**Bu, güvenilir ve kontrol edilebilir AI sistemleri oluşturmak için kritik bir desendir!** 🚀



---

**Feragatname**:  
Bu belge, AI çeviri hizmeti [Co-op Translator](https://github.com/Azure/co-op-translator) kullanılarak çevrilmiştir. Doğruluk için çaba göstersek de, otomatik çeviriler hata veya yanlışlıklar içerebilir. Belgenin orijinal dili, yetkili kaynak olarak kabul edilmelidir. Kritik bilgiler için profesyonel insan çevirisi önerilir. Bu çevirinin kullanımından kaynaklanan yanlış anlamalar veya yanlış yorumlamalardan sorumlu değiliz.
